<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-02-27 14:15:02--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-02-27 14:15:02--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  18.5MB/s    in 69s     

2024-02-27 14:16:12 (20.1 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(word)
    print(wv_embeddings[word])
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

dog
[ 0.6851772  -1.2778991  -0.41913974  1.3623164  -3.1675398   0.09950767
  0.6402681  -1.1245339  -0.6699619  -0.6998852   0.4936771  -0.40500194
 -3.0706816  -2.2809966   0.85798043  2.7093108   0.3492745  -0.03494101
 -0.22330493  1.2290467   1.7755157  -3.158358   -0.6515983   0.7224096
  2.3193083  -1.7969862   0.40903398 -2.744604   -1.7179952  -0.914309
 -0.75887376 -0.35140672 -0.5182776  -1.9097351  -0.8300773   0.02147918
  1.1783471   0.03169126 -0.3069023   1.6666299   0.6711357  -2.1706133
 -0.11800487  0.22336982 -1.2075394  -0.86297905 -0.63865614  1.1733794
  0.10022762  0.7017279   2.7290728  -0.4640484  -2.1719306  -0.3562852
 -1.8449957   0.10270727  1.1125596  -0.8364318   1.9513408  -0.97937447
  1.2650859   0.06809282  0.6477318  -0.52431005 -0.6103959  -2.979829
 -0.7889965  -0.11004248  1.7603841  -1.0547444  -0.98998    -1.1834062
 -2.5359967  -0.35286787  0.7733574   0.16300043  0.76991326  1.9223119
 -1.2843752  -1.5023688   0.15765315 -2.4150877   2.43264

In [ ]:
word = 'steel'
if word in wv_embeddings:
    print(word)
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

steel
float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_simmilar
'''your code'''
similar_words = wv_embeddings.most_similar('dog', topn=5)
for i, (word, similarity) in enumerate(similar_words, start=1):
    print(f"{i}. {word}: {similarity}")

1. animal: 0.8564180135726929
2. dogs: 0.7880866527557373
3. mammal: 0.7623804211616516
4. cats: 0.7621253728866577
5. animals: 0.760793924331665


# **"Cat" (множественное число "cats") в соответствии с эмбеддингами модели входит в пятерку слов похожих на слово  "dog" , и находится на месте 4.**

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    '''your code'''
    # tokenizing the question
    words = tokenizer.tokenize(question)
    # collecting tokens
    vectors = []
    for word in words:
        if word in embeddings:
            vectors.append(embeddings[word])
    # mean vector if words existing in the library
    if vectors:
        question_vec = np.mean(vectors, axis=0)
    else:
        # zero if none is found
        question_vec = np.zeros(dim)
    return question_vec

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
'''your code'''
round(question_to_vec('I love neural networks', wv_embeddings, tokenizer, dim=200)[2], 2)

-1.29

# **-1,29**

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

## **Максимум Hits@47 - DCG@1 может быть достигнут, если дубликат находится среди первых 47 позиций, но не на первой. В этом случае, Hits@47 будет равен 1, и DCG@1 будет равен 0, таким образом, максимальное значение Hits@47 - DCG@1 будет равно:**
# **1−0=1**

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
import numpy as np

# Вычисление DCG@10 для rank_q'i=9
rank_q_i = 9
K = 10

# Вычисление DCG@K
dcg_at_10 = 1 / np.log2(1 + rank_q_i) * (rank_q_i <= K)

dcg_at_10_rounded = round(dcg_at_10, 1)
dcg_at_10_rounded

0.3

# **0,3**

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    # Подсчет количества попаданий, где ранг дубликата <= K
    hits_value = sum(1 for rank in dup_ranks if rank <= k) / len(dup_ranks)
    return hits_value

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    dcg_value = sum(1 / np.log2(1 + rank) for rank in dup_ranks if rank <= k) / len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Colab_Notebooks/DLS_Stepik/13_Intro_to_NLP/stackoverflow_similar_questions.zip

Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        processed_line = line.strip().split()
        data.append(processed_line)
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    question_vec = question_to_vec(question, embeddings, tokenizer, dim).reshape(1, -1)  # Преобразование в 2D массив

    distances = []

    for i, candidate in enumerate(candidates):
        candidate_vec = question_to_vec(candidate, embeddings, tokenizer, dim).reshape(1, -1)  # Преобразование в 2D массив
        # Использование cosine_similarity, которое возвращает массив массивов
        distance = cosine_similarity(question_vec, candidate_vec)[0][0]
        distances.append((1 - distance, i, candidate))  # 1 - distance, т.к. cosine_similarity измеряет сходство, а не расстояние

    distances.sort()
    ranked_candidates = [(i, candidate) for _, i, candidate in distances]

    return ranked_candidates

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
# results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (*, 'select2 not displaying search results'), #скрыт
#             (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?)

# **102**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')
# train_data

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
'''your code'''
combined_questions = [' '.join(pair[0]) + ' <SEP> ' + ' '.join(pair[1]) for pair in train_data]
words = [tokenizer.tokenize(question.lower()) for question in combined_questions]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                    vector_size=200,                 # embedding vector size
                    min_count=5,             # consider words that occured at least 5 times
                    window=5).wv

### **Выбор окна**
**min_count=5: Это уменьшает размер словаря модели, исключая очень редкие слова, что может помочь снизить шум и ускорить обучение.**

**window=5: Большее окно позволяет модели улавливать более широкие контекстные зависимости, что полезно для понимания семантических отношений в вопросах.**

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


### Выводы по семинару и заданным вопросам

- **Принцип токенизации** играет ключевую роль в обработке и анализе текста, поскольку определяет, как текст будет разделён на составляющие (токены) для дальнейшей обработки. Качество токенизации напрямую влияет на результаты модели, так как точное разделение на слова, предложения или другие единицы текста позволяет более корректно анализировать и понимать семантику текста.
Лучше всего с задачей справляется токенизация, которая учитывает контекст и особенности языка (например, использование WordPiece или BPE алгоритмов), так как она позволяет более точно определять границы слов и учитывать морфологические и синтаксические особенности языка.

- **Нормализация слов** включая стемминг и лемматизацию, помогает уменьшить разнообразие форм слова до их базовой формы, что может значительно улучшить качество модели за счёт сокращения размера словаря и уменьшения шума.
Однако, стоит отметить, что в некоторых случаях агрессивная нормализация может привести к потере важной информации о временах, числах и так далее, что потенциально может снизить качество модели в задачах, где такая информация критична.

- **Предобученные эмбеддинги** часто справляются с задачей лучше, поскольку они обучены на больших и разнообразных корпусах данных, что позволяет им лучше улавливать семантические и синтаксические отношения между словами.
С другой стороны, специализированные эмбеддинги, обученные на конкретном наборе данных (например, вопросах StackOverflow), могут быть более эффективными в конкретной предметной области за счёт учета специфики языка и терминологии.

- **Плохое качество решения задачи** может быть связано с несколькими факторами, включая недостаточный объём или качество обучающих данных, неправильный выбор параметров модели, недостаточную предобработку данных и игнорирование контекста и семантики языка. Также важную роль играет сложность самой задачи, так как поиск схожих по смыслу предложений требует глубокого понимания семантики и контекста, что является непростой задачей для автоматических систем.

- **Для улучшения предлагается:**
    - Использование более сложных NLP моделей, таких как трансформеры (BERT, GPT), которые лучше улавливают контекст и семантику текста.
    - Обучение и тонкая настройка моделей на специализированных наборах данных для улучшения их способности понимать специфику задачи и предметной области.
    - Эксперименты с различными стратегиями предобработки данных, чтобы найти оптимальный баланс между универсальностью и специализацией модели.
